<a href="https://colab.research.google.com/github/ll1195831146/Infor7374-AI/blob/master/Assignment1/7374%20Assignment1%20Part1(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this file, I test my best model on GPU.**

In [0]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os
from keras import metrics

In [0]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [23]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20

In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [28]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.,  # set range for random shear
    zoom_range=0.,  # set range for random zoom
    channel_shift_range=0.,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)
    
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

Using real-time data augmentation.


In [29]:
model_best = Sequential()
model_best.add(Conv2D(32, (3, 3), padding='same',
input_shape=x_train.shape[1:]))
model_best.add(Activation('relu'))
model_best.add(Conv2D(32, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Conv2D(64, (3, 3), padding='same'))
model_best.add(Activation('relu'))
model_best.add(Conv2D(64, (3, 3)))
model_best.add(Activation('relu'))
model_best.add(MaxPooling2D(pool_size=(2, 2)))
model_best.add(Dropout(0.1))

model_best.add(Flatten())
model_best.add(Dense(512))
model_best.add(Activation('relu'))
model_best.add(Dropout(0.1))
model_best.add(Dense(num_classes))
model_best.add(Activation('softmax'))

model_best.compile(loss='categorical_crossentropy',
optimizer=keras.optimizers.rmsprop(lr=0.0002, decay=1e-6),
metrics=['accuracy',metrics.categorical_accuracy])

model_best.fit_generator(datagen.flow(x_train, y_train,
    batch_size=batch_size),
    epochs=100,
    steps_per_epoch=1563,
    validation_data=(x_test, y_test),
    workers=4)

Epoch 1/100
1563/1563 [==============================] - 47s 30ms/step - loss: 1.6890 - acc: 0.3844 - categorical_accuracy: 0.3844 - val_loss: 1.4496 - val_acc: 0.4672 - val_categorical_accuracy: 0.4672
Epoch 2/100
1563/1563 [==============================] - 45s 29ms/step - loss: 1.3604 - acc: 0.5105 - categorical_accuracy: 0.5105 - val_loss: 1.1798 - val_acc: 0.5768 - val_categorical_accuracy: 0.5768
Epoch 3/100
1563/1563 [==============================] - 45s 29ms/step - loss: 1.2024 - acc: 0.5697 - categorical_accuracy: 0.5697 - val_loss: 1.0675 - val_acc: 0.6235 - val_categorical_accuracy: 0.6235
Epoch 4/100
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0958 - acc: 0.6136 - categorical_accuracy: 0.6136 - val_loss: 1.0462 - val_acc: 0.6319 - val_categorical_accuracy: 0.6319
Epoch 5/100
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0194 - acc: 0.6392 - categorical_accuracy: 0.6392 - val_loss: 0.9189 - val_acc: 0.6810 - val_categorical_ac

In [30]:
score = model_best.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 1s 150us/step
Test loss: 0.6962530586719513
Test accuracy: 0.7695


**The training efficiency on GPU is much higher than on CPU.**